# Dressmaker - Medium

In [1]:
import $ivy.`org.apache.spark::spark-sql:3.4.0`

import org.apache.log4j.{Level, Logger}
Logger.getLogger("org").setLevel(Level.OFF)

import org.apache.spark._
import org.apache.spark.sql._
import org.apache.spark.sql.types._
import org.apache.spark.sql.functions._
import org.apache.spark.sql.expressions.Window

val spark = {
    NotebookSparkSession.builder()
    .progress(false)
    .appName("app17-2")
    // .master("spark://192.168.31.31:7077")
    .master("local[*]")
    .config("spark.sql.warehouse.dir", 
            "hdfs://192.168.31.31:9000/user/hive/warehouse") 
    .config("spark.cores.max", "4") 
    .config("spark.executor.instances", "1") 
    .config("spark.executor.cores", "2") 
    .config("spark.executor.memory", "10g") 
    .config("spark.shuffle.service.enabled", "false") 
    .config("spark.dynamicAllocation.enabled", "false") 
    .config("spark.sql.catalogImplementation", "hive")
    .config("spark.sql.repl.eagerEval.enabled", "true")
    .config("spark.driver.allowMultipleContexts", "true")
    .getOrCreate()
}

Loading spark-stubs, spark-hive
Adding Hive conf dir /opt/hive/conf to classpath
Creating SparkSession


SLF4J: No SLF4J providers were found.
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See https://www.slf4j.org/codes.html#noProviders for further details.


Spark UI

import $ivy.$                                  


import org.apache.log4j.{Level, Logger}

import org.apache.spark._

import org.apache.spark.sql._

import org.apache.spark.sql.types._

import org.apache.spark.sql.functions._

import org.apache.spark.sql.expressions.Window


spark: SparkSession = org.apache.spark.sql.SparkSession@7fed1f9a

In [2]:
import spark.implicits._
import spark.sqlContext.implicits._
def sc = spark.sparkContext
val hiveCxt = new org.apache.spark.sql.hive.HiveContext(sc)

import spark.implicits._

import spark.sqlContext.implicits._

defined function sc
hiveCxt: sql.hive.HiveContext = org.apache.spark.sql.hive.HiveContext@2361e992

In [3]:
// Credit to Aivean
implicit class RichDF(val ds:DataFrame) {
    def showHTML(limit: Int = 50, truncate: Int = 100) = {
        import xml.Utility.escape
        val data = ds.take(limit)
        val header = ds.schema.fieldNames.toSeq        
        val rows: Seq[Seq[String]] = data.map { row =>
          row.toSeq.map {cell =>
            val str = cell match {
              case null => "null"
              case binary: Array[Byte] => binary.map("%02X".format(_)).mkString("[", " ", "]")
              case array: Array[_] => array.mkString("[", ", ", "]")
              case seq: Seq[_] => seq.mkString("[", ", ", "]")
              case _ => cell.toString
            }
            if (truncate > 0 && str.length > truncate) {
              // do not show ellipses for strings shorter than 4 characters.
              if (truncate < 4) str.substring(0, truncate)
              else str.substring(0, truncate - 3) + "..."
            } else {
              str
            }
          }: Seq[String]
        }
    publish.html(s""" <table>
                <tr>
                 ${header.map(h => s"<th>${escape(h)}</th>").mkString}
                </tr>
                ${rows.map {row =>
                  s"<tr>${row.map{c => s"<td>${escape(c)}</td>" }.mkString}</tr>"
                }.mkString}
            </table>
        """)
    }
}

defined class RichDF

In [4]:
val jmcust = hiveCxt.table("sqlzoo.jmcust")
val dressmaker = hiveCxt.table("sqlzoo.dressmaker")
val dress_order = hiveCxt.table("sqlzoo.dress_order")
val construction = hiveCxt.table("sqlzoo.construction")
val quantities = hiveCxt.table("sqlzoo.quantities")
val order_line = hiveCxt.table("sqlzoo.order_line")
val garment = hiveCxt.table("sqlzoo.garment")
val material = hiveCxt.table("sqlzoo.material")

jmcust: DataFrame = [c_no: int, c_name: string ... 2 more fields]
dressmaker: DataFrame = [d_no: int, d_name: string ... 2 more fields]
dress_order: DataFrame = [order_no: int, cust_no: int ... 2 more fields]
construction: DataFrame = [maker: int, order_ref: int ... 3 more fields]
quantities: DataFrame = [style_q: int, size_q: int ... 1 more field]
order_line: DataFrame = [order_ref: int, line_no: int ... 3 more fields]
garment: DataFrame = [style_no: int, description: string ... 2 more fields]
material: DataFrame = [material_no: int, fabric: string ... 3 more fields]

## 1.
Assuming that any garment could be made in any of the available materials, list the garments (description, fabric, colour and pattern) which are expensive to make, that is, those for which the labour costs are 80% or more of the total cost.

In [5]:
(garment.join(material)
 .filter(col("labour_cost")/(col("labour_cost")+col("cost"))>0.8) 
 .select("description", "fabric", "colour", "pattern")
 .showHTML())

description,fabric,colour,pattern
Trousers,Cotton,Yellow Stripe,Woven
Long Skirt,Cotton,Yellow Stripe,Woven
Short Skirt,Cotton,Yellow Stripe,Woven
Sundress,Cotton,Yellow Stripe,Woven
Trousers,Cotton,Green Stripe,Woven
Long Skirt,Cotton,Green Stripe,Woven
Short Skirt,Cotton,Green Stripe,Woven
Sundress,Cotton,Green Stripe,Woven
Trousers,Cotton,Black Dotted,Woven
Long Skirt,Cotton,Black Dotted,Woven


## 2.
List the descriptions and the number of orders of the less popular garments, that is those for which less than the average number of orders per garment have been placed. Also print out the average number of orders per garment. When calculating the average, ignore any garments for which no orders have been made.

In [6]:
(garment.join(order_line, (garment("style_no") === order_line("ol_style")))
 .groupBy("style_no", "description")
 .agg(count("order_ref").alias("n_orders"))
 .withColumn("avg_orders", avg(col("n_orders")).over(Window.partitionBy(lit(0))))
 .filter(col("n_orders") < col("avg_orders"))
 .select("description", "n_orders", "avg_orders")
 .showHTML()
)

description,n_orders,avg_orders
Sundress,5,5.166666666666667
Short Skirt,5,5.166666666666667
Shorts,5,5.166666666666667
Suntop,4,5.166666666666667


## 3.
Which is the most popular line, that is, the garment with the highest number of orders. Bearing in mind the fact that there may be several such garments, list the garment description(s) and number(s) of orders.

In [7]:
(garment.join(order_line, (garment("style_no") === order_line("ol_style")))
 .groupBy("style_no", "description")
 .agg(count("order_ref").alias("n_orders"))
 .withColumn("max_orders", max("n_orders").over(Window.partitionBy(lit(0))))
 .filter(col("n_orders") === col("max_orders"))
 .select("description", "n_orders")
 .showHTML())

description,n_orders
Trousers,6
Long Skirt,6


## 4.
List the descriptions, and costs of the more expensive size 8, Cotton garments which might be ordered, that is those costing more than the average (labour costs + material costs) to make.

In [9]:
(garment.join(order_line, (garment("style_no") === order_line("ol_style")))
 .join(material, (order_line("ol_material") === material("material_no")))
 .join(quantities, ((order_line("ol_size") === quantities("size_q")) &&
                    (order_line("ol_style") === quantities("style_q"))))
 .withColumn("tot_cost", round(col("labour_cost") + col("quantity") * col("cost"), 2))
 .withColumn("avg_cost", avg(col("tot_cost")).over(Window.partitionBy(lit(0))))
 .filter((col("ol_size") === 8) && (lower(trim(col("fabric"))) === "cotton") &&
         (col("tot_cost") > col("avg_cost")))
 .select("description", "material_no", "tot_cost")
 .showHTML())

description,material_no,tot_cost
Sundress,14,31.2


## 5.
What is the most common size ordered for each garment type? List description, size and number of orders, assuming that there could be several equally popular sizes for each type.

In [10]:
(garment.join(order_line, (garment("style_no") === order_line("ol_style")))
 .groupBy("style_no", "description", "ol_size")
 .agg(count("order_ref").alias("n_orders"))
 .groupBy("style_no", "description", "ol_size")
 .max("n_orders")
 .orderBy("style_no", "description", "ol_size")
 .showHTML())

style_no,description,ol_size,max(n_orders)
1,Trousers,8,3
1,Trousers,14,1
1,Trousers,16,1
1,Trousers,18,1
2,Long Skirt,8,3
2,Long Skirt,14,1
2,Long Skirt,16,1
2,Long Skirt,18,1
3,Shorts,8,1
3,Shorts,10,1


In [11]:
spark.stop()